# Quadratic Test

In [209]:
import scipy as sc
from sklearn import datasets
from matplotlib.style import available
from sklearn.cluster import KMeans
import numpy as np
import cvxpy as cp
import torch
import time
from cvxpylayers.torch import CvxpyLayer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings
import mosek
warnings.filterwarnings('ignore')
plt.rcParams.update({
    "text.usetex":True,
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive","tab:blue", "tab:orange", "tab:green",
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive"]

In [210]:
def createproblem_quad(n, P_inv):
    # PARAMETERS #
    A = cp.Parameter((n,n))
    b = cp.Parameter(n)
    # VARIABLES #
    # weights, s_i, lambda, tau
    x = cp.Variable(n)
    t = cp.Variable()
    z = cp.Variable(n)
    y = cp.Variable((n,n))
    gam = cp.Variable(n)
    # OBJECTIVE #
    objective = t

    # CONSTRAINTS #
    constraints = [cp.sum([cp.quad_over_lin(y[i]@P_inv[i],2*x[i]) for i in range(n)]) + b@z + cp.norm(gam,2) <= t]
    constraints += [A.T@gam == z]
    constraints += [cp.sum(y, axis = 1) == z]
    constraints += [cp.sum(x) == 1]
    constraints += [x >= 0, x <= 1]
    # PROBLEM #
    problem = cp.Problem(cp.Minimize(objective), constraints)
    return problem, x, t, A, b

def normal_returns(m, N):
    R = np.vstack([np.random.normal(
        i*0.03, np.sqrt((0.02**2+(i*0.025)**2)), N) for i in range(1, m+1)])
    return (R.transpose())
def normal_returns1(m, N):
    R = np.vstack([np.random.normal(
        i*0.05, np.sqrt((0.01**2+(i*0.035)**2)), N) for i in range(1, m+1)])
    return (R.transpose())
def loss(x,t,data,l = 1): 
    cur_sum = 0
    for i in range(data.shape[0]):
        cur_sum = cur_sum + torch.maximum(torch.sum(torch.tensor([-0.5*data[i]@torch.tensor(P[ind],requires_grad = True)
                                                                  *x[ind]@data[i] for ind in range(m)], requires_grad = True)) - t, 
                                          torch.tensor(0., requires_grad=True))
    return t + l*cur_sum/data.shape[0], t, cur_sum/data.shape[0]
                                                                                                                                                                                                                                                                                                 

In [211]:
loss(torch.tensor(5*np.ones(m)),torch.tensor(1),val_dset)

(tensor(1., dtype=torch.float64, grad_fn=<AddBackward0>),
 tensor(1),
 tensor(0., dtype=torch.float64, grad_fn=<DivBackward0>))

In [222]:
P = {}
P_inv = {}
m = 15
N = 100
for i in range(m):
    P[i] = datasets.make_spd_matrix(m, random_state=i)
    P_inv[i] = sc.linalg.sqrtm(np.linalg.inv(P[i]))
d1 = normal_returns(m,N)
d2 = normal_returns1(m,N)
data = np.concatenate((d1,d2))

In [223]:
w = 5
problem, x,t,A,b = createproblem_quad(m, P_inv)
df1 = pd.DataFrame(columns=["step", "Opt_val", "Violations1", "Violations2","Eval_val", "Loss_val", "A_norm"])
train, test = train_test_split(data, test_size=int(data.shape[0]/5), random_state=16)
val_dset = torch.tensor(train, requires_grad=True)
eval_set = torch.tensor(test, requires_grad=True)
# create cvxpylayer
cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables= [x, t])

init = sc.linalg.sqrtm(sc.linalg.inv(np.cov(train.T)))
paramb_tch = torch.tensor(-init@np.mean(train, axis=0), requires_grad=True)
paramT_tch = torch.tensor(init, requires_grad=True)
variables = [paramT_tch, paramb_tch]
opt = torch.optim.SGD(variables, lr=0.01, momentum=0.8)
step = 50
for steps in range(step):
    var_values = cvxpylayer(paramT_tch, paramb_tch, solver_args={'solve_method': 'ECOS'})
    totloss, obj, v1 = loss(*var_values, val_dset, l= w)
    evalloss, _, v2 = loss(*var_values, eval_set, l = w)
    totloss.backward()
    newrow = pd.Series(
        {"step": steps,
         "Loss_val": totloss.item(),
         "Eval_val": evalloss.item(),
         "Opt_val": obj.item(),
         "Violations1": v1.item(),
         "Violations2": v2.item(),
         "A_norm": np.linalg.norm(paramT_tch.detach().numpy().copy())
         })
    df1 = pd.concat([df1, newrow.to_frame().T], ignore_index=True)
    if steps < step - 1:
        opt.step()
        opt.zero_grad()
        
problem, x,t,A,b = createproblem_quad(m, P_inv)
cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables= [x, t])       
df2 = pd.DataFrame(columns=["step", "Opt_val", "Violations1", "Violations2","Eval_val", "Loss_val", "A_norm"])
init = np.eye(m)
paramb_tch = torch.tensor(-init@np.mean(train, axis=0), requires_grad=True)
paramT_tch = torch.tensor(init, requires_grad=True)
variables = [paramT_tch, paramb_tch]
opt = torch.optim.SGD(variables, lr=0.01, momentum=0.8)
step = 50
for steps in range(step):
    var_values = cvxpylayer(paramT_tch, paramb_tch, solver_args={'solve_method': 'ECOS'})
    totloss, obj, v1 = loss(*var_values, val_dset, l=w)
    evalloss, _, v2 = loss(*var_values, eval_set, l=w)
    totloss.backward()
    newrow = pd.Series(
        {"step": steps,
         "Loss_val": totloss.item(),
         "Eval_val": evalloss.item(),
         "Opt_val": obj.item(),
         "Violations1": v1.item(),
         "Violations2": v2.item(),
         "A_norm": np.linalg.norm(paramT_tch.detach().numpy().copy())
         })
    df2 = pd.concat([df2, newrow.to_frame().T], ignore_index=True)
    if steps < step - 1:
        opt.step()
        opt.zero_grad()
problem, x,t,A,b = createproblem_quad(m, P_inv)
cvxpylayer = CvxpyLayer(problem, parameters=[A, b], variables= [x, t])  
df_eps = pd.DataFrame(columns=["step", "Opt_val", "Violations1", "Violations2","Eval_val", "Loss_val", "A_norm"])
eps_tch = torch.tensor(1/1, requires_grad=True)
paramb_tch = eps_tch*torch.tensor(-np.mean(train, axis=0), requires_grad=True)
paramT_tch = eps_tch*torch.tensor(np.eye(train.shape[1]), requires_grad=True)
variables = [eps_tch]
opt = torch.optim.SGD(variables, lr=0.01, momentum=0.8)
step = 50
for steps in range(step):
    paramb_tch = eps_tch*torch.tensor(-np.mean(train, axis=0), requires_grad=True)
    paramT_tch = eps_tch*torch.tensor(np.eye(train.shape[1]), requires_grad=True)
    var_values = cvxpylayer(paramT_tch, paramb_tch, solver_args={'solve_method': 'ECOS'})
    totloss, obj, v1 = loss(*var_values, val_dset, l=w)
    evalloss, _, v2 = loss(*var_values, eval_set, l=w)
    totloss.backward()
    newrow = pd.Series(
        {"step": steps,
         "Loss_val": totloss.item(),
         "Eval_val": evalloss.item(),
         "Opt_val": obj.item(),
         "Violations1": v1.item(),
         "Violations2": v2.item(),
         "A_norm": eps_tch.detach().numpy().copy()
        })
    df_eps = pd.concat([df_eps, newrow.to_frame().T], ignore_index=True)
    if steps < step - 1:
        opt.step()
        opt.zero_grad()

In [224]:
plt.figure(figsize=(9, 5))
plt.plot(df_eps['step'], df_eps['Loss_val'], color="tab:blue", label=r"$\epsilon$ training")
plt.plot(df_eps['step'], df_eps['Eval_val'], linestyle='--', color="tab:blue", label=r"$\epsilon$ testing")
# plt.plot(df1['step'], df1['Loss_val'], color="tab:orange", label="Reshape training init $\Sigma^{-1/2}$")
# plt.plot(df1['step'], df1['Eval_val'], linestyle='--', color="tab:orange", label="Reshape testing init $\Sigma^{-1/2}$")
plt.plot(df2['step'], df2['Loss_val'], color="tab:green", label="Reshape training init $I$")
plt.plot(df2['step'], df2['Eval_val'], linestyle='--', color="tab:green", label="Reshape testing init $I$")
plt.legend(loc="lower right", bbox_to_anchor=(1.6, 0.2))
plt.xlabel("Iterations")
plt.ylabel("Training and Testing Loss")
plt.savefig("plot.pdf")


In [199]:
df_eps.to_csv('df_eps.csv')
df2.to_csv('df_R_I.csv')
df1.to_csv('df_R_C.csv')

In [221]:
df_eps

,step,Opt_val,Violations1,Violations2,Eval_val,Loss_val,A_norm
0,0,-0.167884,0.0,0.0,-0.167884,-0.167884,1.0
1,1,-0.206825,0.0,0.0,-0.206825,-0.206825,1.019239
2,2,-0.291613,0.0,0.0,-0.291613,-0.291613,1.055871
3,3,-0.443507,0.001985,0.000411,-0.44145,-0.433582,1.110233
4,4,-0.686957,0.013876,0.016657,-0.603672,-0.617577,1.1807393
5,5,-1.028322,0.06258,0.06488,-0.70392,-0.71542,1.2610352
6,6,-1.337795,0.145449,0.147241,-0.601591,-0.610548,1.3223374
7,7,-1.45943,0.186225,0.189558,-0.51164,-0.528305,1.3443513
8,8,-1.32313,0.141014,0.142577,-0.610248,-0.61806,1.319617
9,9,-1.08507,0.075639,0.075503,-0.707555,-0.706873,1.2729485
